In [1]:
import crocoddyl
import pinocchio
from IPython.display import HTML
from mim_solvers import SolverSQP
import numpy as np
import random
import pinocchio as pin
from matplotlib import animation
from matplotlib import pyplot as plt
from PointMass_utils import *
# from PointMass_model import DifferentialActionModelPointMass
# from pinocchio.robot_wrapper import RobotWrapper
# from robot_descriptions.loaders.pinocchio import load_robot_description

In [2]:
# pm = RobotWrapper.BuildFromURDF('PM_model.urdf', None, pin.JointModelFreeFlyer())
# pm_model = pm.model
# pm_data = pm.data
# pm_model.gravity = pin.Motion.Zero()


In [3]:
# pm_model.effortLimit

In [4]:
# q_rand = pin.randomConfiguration(pm_model)
# print(q_rand)
# pin.forwardKinematics(pm_model, pm_data, q_rand, np.zeros(2))
# pin.framesForwardKinematics(pm_model, pm_data, q_rand)
# pin.updateFramePlacements(pm_model, pm_data)  
# print(pm_model.jointPlacements.tolist())

In [5]:
nx = 4
nu = 2
T = 30
timeStep = 5e-2
start_point = np.array([0, 0, 0, 0])
target_low = np.array([10, 0, 0, 0])
target_high = np.array([10, 10, 0, 0])
obs0 = Obstacle(5, 5, 2, 'Obs0')
obs1 = Obstacle(3.5, 3.5, 1, 'Obs1')
obs2 = Obstacle(6.5, 6.5, 1, 'Obs2')
obs3 = Obstacle(6.5, 3.5, 1, 'Obs3')
obs_set = [obs1, obs2, obs3]
# obs_set = []

In [6]:
# Custom 2-Norm Activation Model for the 2D obstacle avoidance case
class ActivationModel2NormCustom(crocoddyl.ActivationModelAbstract):
    def __init__(self, state, R, nr):
        crocoddyl.ActivationModelAbstract.__init__(self, nr)
        self.d = 0.0
        self.state = state
        self.alpha = R
        
    def calc(self, data, r):
        self.d = np.linalg.norm(r[0:self.state.nq])
        if self.d > self.alpha:
            data.a_value = 0.0
        else:
            data.a_value = 0.5 * np.sum((self.d - self.alpha)**2)
        
    def calcDiff(self, data, r):
        if self.d < self.alpha:
            data.Ar = (self.d - self.alpha)/self.d * r
            if self.true_hessian:
                diag_values = np.einsum('ii->i',data.Arr)
                diag_values = self.alpha * r**2 / self.d**3
                diag_values += (self.d - self.alpha) / self.d
            else:
                diag_values = np.einsum('ii->i',data.Arr)
                diag_values = r**2 / self.d**2
        else:
            data.Ar = np.zeros_like(data.Ar)
            data.Arr = np.zeros_like(data.Arr)
            
        

In [7]:
pm_model = pin.buildModelFromUrdf('PM_model.urdf')
pm_model.gravity = pin.Motion.Zero()
pm_data = pm_model.createData()
q_rand = pin.randomConfiguration(pm_model)

In [8]:
pm_model.jointPlacements.tolist()

[SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]]))]

In [9]:
# print(q_rand)
# pin.forwardKinematics(pm_model, pm_data, q_rand, np.zeros(nq))
# pin.framesForwardKinematics(pm_model, pm_data, q_rand)
# pin.updateFramePlacements(pm_model, pm_data)  
# pm_model.jointPlacements.tolist()

In [10]:
# pin.computeJointJacobians(pm_model, pm_data)

In [11]:
nq = pm_model.nq; nv = nq
nu = 2; nx = nq + nv
# state = crocoddyl.StateVector(nx)
state = crocoddyl.StateMultibody(pm_model)
stateDataCollector = crocoddyl.DataCollectorAbstract()
actuation = crocoddyl.ActuationModelAbstract(state, nu)
actuationData = crocoddyl.ActuationDataAbstract(actuation)
actuationDataCollector = crocoddyl.DataCollectorActuation(actuationData)

# U-Reg
uResidual = crocoddyl.ResidualModelControl(state)
uRegCost = crocoddyl.CostModelResidual(state, uResidual)
uRegCostData = uRegCost.createData(actuationDataCollector)

# X-Reg
xResidual = crocoddyl.ResidualModelState(state, start_point)
xRegCost = crocoddyl.CostModelResidual(state, xResidual)
xRegCostData = xRegCost.createData(stateDataCollector)

# Goal Translation
transResidual = crocoddyl.ResidualModelState(state, target_high)
transResCost = crocoddyl.CostModelResidual(state, transResidual)
transResCostData = transResCost.createData(stateDataCollector)

# Obstacles
collisionCost = []
collisionData = []
for i, obs in enumerate(obs_set):
    collisionActivation = ActivationModel2NormCustom(state, obs.R, nr = 4)
    collisionResidual = crocoddyl.ResidualModelState(state, np.array([obs.x, obs.y, 0.0, 0.0]))
    collisionCost.append(crocoddyl.CostModelResidual(state, collisionActivation, collisionResidual))
    collisionData.append(collisionCost[i].createData(stateDataCollector))

In [12]:
# Setting Weights
w_running = {}
w_terminal = {}

w_running['XReg'] = 1e-1;          w_terminal['XReg'] = 1e2
w_running['UReg'] = 1e-4;          w_terminal['UReg'] = 1e2
w_running['translation'] = 1e1;    w_terminal['translation'] = 1e2
w_running['Obs'] = 1e-1;           w_terminal['Obs'] = 1e3

In [13]:
# Adding Costs
runningCostModel = crocoddyl.CostModelSum(state)
terminalCostModel = crocoddyl.CostModelSum(state)
for w, model in zip([w_running, w_terminal],[runningCostModel, terminalCostModel]):
    model.addCost("XReg", xRegCost, w['XReg'])
    model.addCost("UReg", uRegCost, w['UReg'])
    model.addCost("translation", transResCost, w['translation'])
    for c, colCost in enumerate(collisionCost):
        model.addCost("obs"+str(c), colCost, w['Obs'])

In [14]:
# NumDiff Differential Analytical Model
# running_DAM = crocoddyl.DifferentialActionModelNumDiff(runningCostModel, False)
# terminal_DAM = crocoddyl.DifferentialActionModelNumDiff(terminalCostModel, False)

In [15]:
# Differential Analytical Model
running_DAM = crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuation, runningCostModel)
terminal_DAM = crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuation, terminalCostModel)

In [16]:
# running_DAM.costs.costs['obs1'].cost.activation.alpha

In [17]:
running_DAM.costs.costs.todict()

{'UReg': {w=0.0001, CostModelResidual {ResidualModelControl, ActivationModelQuad {nr=2}}},
 'XReg': {w=0.1, CostModelResidual {ResidualModelState, ActivationModelQuad {nr=4}}},
 'obs0': {w=0.1, CostModelResidual {ResidualModelState, crocoddyl::python::ActivationModelAbstract_wrap}},
 'obs1': {w=0.1, CostModelResidual {ResidualModelState, crocoddyl::python::ActivationModelAbstract_wrap}},
 'obs2': {w=0.1, CostModelResidual {ResidualModelState, crocoddyl::python::ActivationModelAbstract_wrap}},
 'translation': {w=10, CostModelResidual {ResidualModelState, ActivationModelQuad {nr=4}}}}

In [18]:
# Create Integrated Action Model (IAM), i.e. Euler integration of continuous dynamics and cost
runningModel = crocoddyl.IntegratedActionModelEuler(running_DAM, timeStep)
terminalModel = crocoddyl.IntegratedActionModelEuler(terminal_DAM, 0.)

In [21]:
x0 = np.zeros(state.nx)
u0 = np.zeros(actuation.nu)
problem = crocoddyl.ShootingProblem(x0, [runningModel] * T, terminalModel)
sqp_opt = SolverSQP(problem)
sqp_opt.solve
# solver.setCallbacks([crocoddyl.CallbackLogger(), crocoddyl.CallbackVerbose()])
# Warm start : initial state + gravity compensation
xs_init = [x0 for i in range(T+1)]
us_init = [u0 for i in range(T)]

In [ ]:
us_init

In [23]:
sqp_opt.solve()

TypeError: 'NoneType' object is not callable